# Test Full Pipeline: End-to-End (M11)

This notebook runs the complete YOLO-OBB pipeline end-to-end:

```
image -> detect -> crop -> rotate+OCR -> parse -> normalize -> validate -> expand -> match -> score
```

Uses `YOLOPipeline` which orchestrates all 10 stages in a single `run()` call.

**Runtime requirement:** GPU with at least 8 GB VRAM (A100 preferred).

**Required files on Google Drive:**
- YOLO-OBB model weights (`best.pt`)
- Sample page image (PNG/JPG)
- SolidWorks JSON export
- HuggingFace token for LightOnOCR-2

In [ ]:
# Cell 1: Install all dependencies
# NOTE: Set your runtime to GPU before running (Runtime > Change runtime type > A100)
%pip install ultralytics transformers torch pillow accelerate --quiet

# Clone the repo
!git clone https://github.com/skaumbdoallsaws-coder/AI-Drawing-Inspector.git /content/AI-Drawing-Inspector 2>/dev/null || \
    (cd /content/AI-Drawing-Inspector && git pull)

print('Dependencies installed.')

In [ ]:
# Cell 2: Mount Drive, set all paths
import sys
import os
from google.colab import drive, userdata

drive.mount('/content/drive')
sys.path.insert(0, '/content/AI-Drawing-Inspector')

# ---- HuggingFace token ----
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
except Exception:
    HF_TOKEN = None
# HF_TOKEN = 'hf_YOUR_TOKEN_HERE'  # Uncomment for manual entry

if HF_TOKEN:
    os.environ['HF_TOKEN'] = HF_TOKEN
    print(f'HF_TOKEN set (length={len(HF_TOKEN)})')
else:
    print('WARNING: HF_TOKEN not set.')

# ---- Paths (edit to match your Drive layout) ----
MODEL_PATH = '/content/drive/MyDrive/ai_inspector_models/best.pt'
SAMPLE_IMAGE = '/content/drive/MyDrive/ai_inspector_data/sample_pages/test_page.png'
SW_JSON_PATH = '/content/drive/MyDrive/ai_inspector_data/sw_json/test_part.json'
OUTPUT_DIR = '/content/debug/pipeline_run'

# Title block text (paste from OCR or manual inspection)
TITLE_BLOCK_TEXT = 'UNLESS OTHERWISE SPECIFIED DIMENSIONS ARE IN INCHES'

os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f'Model:       {MODEL_PATH}  exists={os.path.exists(MODEL_PATH)}')
print(f'Image:       {SAMPLE_IMAGE}  exists={os.path.exists(SAMPLE_IMAGE)}')
print(f'SW JSON:     {SW_JSON_PATH}  exists={os.path.exists(SW_JSON_PATH)}')
print(f'Output dir:  {OUTPUT_DIR}')

In [ ]:
# Cell 3: Import YOLOPipeline
from ai_inspector.pipeline.yolo_pipeline import YOLOPipeline, PipelineResult

print('YOLOPipeline imported successfully.')
print(f'PipelineResult fields: {[f.name for f in PipelineResult.__dataclass_fields__.values()]}')

In [ ]:
# Cell 4: Create and load pipeline (show GPU memory usage)
import torch

def gpu_mem():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1e9
        reserved = torch.cuda.memory_reserved() / 1e9
        total = torch.cuda.get_device_properties(0).total_mem / 1e9
        return f'{allocated:.2f} GB allocated / {reserved:.2f} GB reserved / {total:.1f} GB total'
    return 'No CUDA'

print(f'GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A"}')
print(f'Before load: {gpu_mem()}')

pipeline = YOLOPipeline(
    model_path=MODEL_PATH,
    hf_token=HF_TOKEN,
    confidence_threshold=0.25,
    device='cuda',
)

pipeline.load()
print(f'Pipeline loaded: {pipeline.is_loaded}')
print(f'After load:  {gpu_mem()}')

In [ ]:
# Cell 5: Run pipeline on a test page
import time

start = time.time()

result = pipeline.run(
    image_path=SAMPLE_IMAGE,
    sw_json_path=SW_JSON_PATH,
    title_block_text=TITLE_BLOCK_TEXT,
    page_id='test_page_0',
    output_dir=OUTPUT_DIR,
    save_crops=True,
)

elapsed = time.time() - start
print(f'Pipeline completed in {elapsed:.1f}s')
print(f'Packets: {len(result.packets)}')
print(f'Match results: {len(result.match_results)}')

In [ ]:
# Cell 6: Display results: scores, expansion summary, validation stats
import json

print('=== SCORES ===')
for key, val in result.scores.items():
    print(f'  {key:25s}: {val}')

print('\n=== EXPANSION SUMMARY ===')
print(json.dumps(result.expansion_summary, indent=2))

print('\n=== VALIDATION STATS ===')
print(json.dumps(result.validation_stats, indent=2))

print('\n=== PACKET SUMMARY ===')
print(json.dumps(result.packet_summary, indent=2))

In [ ]:
# Cell 7: Display match results table
from ai_inspector.comparison.matcher import MatchStatus

print(f'{"#":>3s} {"Status":15s} {"Type":18s} {"Delta":>10s} {"Notes"}')
print('=' * 90)

for i, r in enumerate(result.match_results):
    # Determine type from whichever side is present
    callout_type = ''
    if r.drawing_callout:
        callout_type = r.drawing_callout.get('calloutType', '')
    elif r.sw_feature:
        callout_type = r.sw_feature.feature_type

    delta_str = f'{r.delta:+.4f}' if r.delta is not None else 'N/A'

    # Color coding via emoji-free markers
    status_marker = {
        MatchStatus.MATCHED: '[OK]',
        MatchStatus.MISSING: '[MISS]',
        MatchStatus.EXTRA: '[EXTRA]',
        MatchStatus.TOLERANCE_FAIL: '[TOL]',
        MatchStatus.SKIPPED: '[SKIP]',
    }.get(r.status, '[?]')

    print(f'{i:3d} {status_marker + " " + r.status.value:15s} '
          f'{callout_type:18s} {delta_str:>10s} {r.notes[:50]}')

In [ ]:
# Cell 8: Display packet provenance for first 5 detections
from ai_inspector.schemas.callout_packet import packet_to_dict

print('=== Packet Provenance (first 5) ===')
print()

for i, pkt in enumerate(result.packets[:5]):
    print(f'--- Packet {i}: {pkt.det_id} ---')

    # Detection
    if pkt.detection:
        print(f'  Detection: class={pkt.detection.class_name}, '
              f'conf={pkt.detection.confidence:.3f}')

    # Crop
    if pkt.crop:
        meta = pkt.crop.meta
        print(f'  Crop: {meta.get("crop_w", "?")}x{meta.get("crop_h", "?")}px, '
              f'angle={meta.get("rotation_angle", 0):.1f}deg')

    # Rotation
    if pkt.rotation:
        print(f'  Rotation: {pkt.rotation.rotation_used}deg, '
              f'quality={pkt.rotation.quality_score:.2f}')

    # Reader
    if pkt.reader:
        print(f'  Reader: type={pkt.reader.callout_type}, '
              f'source={pkt.reader.source}, '
              f'ocr_conf={pkt.reader.ocr_confidence:.2f}')
        print(f'  Raw: "{pkt.reader.raw[:60]}"')
        parsed_keys = [k for k in pkt.reader.parsed.keys() if not k.startswith('_')]
        print(f'  Parsed fields: {parsed_keys}')

    # Normalization
    if pkt.normalized:
        method = pkt.normalized.get('_normalization_method', '?')
        units = pkt.normalized.get('_detected_units', '?')
        print(f'  Normalization: method={method}, detected_units={units}')

    # Validation
    print(f'  Validated: {pkt.validated}'
          + (f', error="{pkt.validation_error}"' if pkt.validation_error else ''))

    print()

In [ ]:
# Cell 9: Save all artifacts to output dir
import json
from pathlib import Path

out = Path(OUTPUT_DIR)
print(f'Artifacts saved to: {OUTPUT_DIR}/')
print()

# List saved files
for f in sorted(out.rglob('*')):
    if f.is_file():
        size_kb = f.stat().st_size / 1024
        print(f'  {f.relative_to(out)}  ({size_kb:.1f} KB)')

# Save the full result dict as well
result_dict = result.to_dict()
summary_path = out / 'pipeline_summary.json'
with open(summary_path, 'w') as f:
    json.dump(result_dict, f, indent=2, ensure_ascii=False)
print(f'\nPipeline summary saved to: {summary_path}')

In [ ]:
# Cell 10: Unload pipeline, show memory freed
import torch

def gpu_mem():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1e9
        reserved = torch.cuda.memory_reserved() / 1e9
        total = torch.cuda.get_device_properties(0).total_mem / 1e9
        return f'{allocated:.2f} GB allocated / {reserved:.2f} GB reserved / {total:.1f} GB total'
    return 'No CUDA'

print(f'Before unload: {gpu_mem()}')

pipeline.unload()

# Force garbage collection
import gc
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print(f'After unload:  {gpu_mem()}')
print(f'Pipeline loaded: {pipeline.is_loaded}')
print('\nDone. Pipeline unloaded and memory freed.')